<center><img src='https://drive.google.com/uc?id=1_utx_ZGclmCwNttSe40kYA6VHzNocdET' height="60"></center>

AI TECH - Akademia Innowacyjnych Zastosowań Technologii Cyfrowych. Programu Operacyjnego Polska Cyfrowa na lata 2014-2020
<hr>

<center><img src='https://drive.google.com/uc?id=1BXZ0u3562N_MqCLcekI-Ens77Kk4LpPm'></center>

<center>
Projekt współfinansowany ze środków Unii Europejskiej w ramach Europejskiego Funduszu Rozwoju Regionalnego
Program Operacyjny Polska Cyfrowa na lata 2014-2020,
Oś Priorytetowa nr 3 "Cyfrowe kompetencje społeczeństwa" Działanie  nr 3.2 "Innowacyjne rozwiązania na rzecz aktywizacji cyfrowej"
Tytuł projektu:  „Akademia Innowacyjnych Zastosowań Technologii Cyfrowych (AI Tech)”
    </center>

# Logistic regression

In this exercise you will train a logistic regression model via gradient descent in two simple scenarios.

The general setup is as follows:
* we are given a set of pairs $(x, y)$, where $x \in R^D$ is a vector of real numbers representing the features, and $y \in \{0,1\}$ is the target,
* for a given $x$ we model the probability of $y=1$ by $h(x):=g(w^Tx)$, where $g$ is the sigmoid function: $g(z) = \frac{1}{1+e^{-z}}$,
* to find the right $w$ we will optimize the so called logarithmic loss: $J(w) = -\frac{1}{n}\sum_{i=1}^n y_i \log{h(x_i)} + (1-y_i) \log{(1-h(x_i))}$,
* with the loss function in hand we can improve our guesses iteratively:
    * $w_j^{t+1} = w_j^{t} - \eta \cdot \frac{\partial J(w)}{\partial w_j}$

* we can end the process after some predefined number of epochs (or when the changes are no longer meaningful).

Let's start with the simplest example - linear separated points on a plane.

The formula for the partial derivative of the logistic loss function $J(w)$ with respect to a weight $w_j$ is:

$$ \frac{\partial J(w)}{\partial w_j} = \frac{1}{n} \sum_{i=1}^n (h(x_i) - y_i) x_{ij} $$

where:
- $n$ is the number of training examples
- $h(x_i)$ is the predicted probability for the i-th example ($h(x_i) = g(w^Tx_i)$)
- $y_i$ is the true target for the i-th example
- $x_{ij}$ is the j-th feature of the i-th example

This formula can be vectorized as:

$$ \nabla J(w) = \frac{1}{n} X^T (h(X) - y) $$

where:
- $X$ is the matrix of features (with a column of ones for the bias term)
- $y$ is the vector of true targets
- $h(X)$ is the vector of predicted probabilities for all examples

In [2]:
import numpy as np

np.random.seed(123)

# these parametrize the line
a = 0.3
b = -0.2
c = 0.001

# ax + by + c = 0
# y = a/b x - b / c

# True/False mapping
def lin_rule(point, noise=0.):
    '''
    Returns whether the point is above or below the line.
    > 0: left of the line, < 0: below it.
    '''
    return a * point[0] + b * point[1] + c + noise < 0.

# Just for plotting
def get_y_of_x_fun(a, b, c):
    def y(x):
        return - x * a / b - c / b
    return y

lin_fun = get_y_of_x_fun(a, b, c)

In [3]:
# Generate the training data

n = 500
range_points = 1
sigma = 0.05

X = range_points * 2 * (np.random.rand(n, 2) - 0.5)
y = [lin_rule(x, sigma * np.random.normal()) for x in X]

print(X[:10])
print(y[:10])

[[ 0.39293837 -0.42772133]
 [-0.54629709  0.10262954]
 [ 0.43893794 -0.15378708]
 [ 0.9615284   0.36965948]
 [-0.0381362  -0.21576496]
 [-0.31364397  0.45809941]
 [-0.12285551 -0.88064421]
 [-0.20391149  0.47599081]
 [-0.63501654 -0.64909649]
 [ 0.06310275  0.06365517]]
[np.False_, np.True_, np.False_, np.False_, np.False_, np.True_, np.False_, np.True_, np.True_, np.False_]


Let's plot the data.

In [4]:
import plotly.express as px

# plotly has a problem with coloring boolean values, hence stringify
# see https://community.plotly.com/t/plotly-express-scatter-color-not-showing/25962
fig = px.scatter(x=X[:, 0], y=X[:, 1], color=list(map(str, y)))
x_range = [np.min(X[:, 0]), np.max(X[:, 1])]
fig.add_scatter(x=x_range, y=list(map(lin_fun, x_range)), name='ground truth border')
fig.show()

Now, let's implement and train a logistic regression model. You should obtain accuracy of at least 96%.

In [5]:
import numpy as np
X = np.array(X)
y = np.array(y)

In [6]:
# Generate the training data

print(n, X.shape, y.shape)

500 (500, 2) (500,)


In [7]:
def sigmoid(x: np.array) -> np.array:
  return 1 / (1 + np.e ** -x)
sigmoid(np.array([-3, -2, -1, 0, 1,2,3]))

array([0.04742587, 0.11920292, 0.26894142, 0.5       , 0.73105858,
       0.88079708, 0.95257413])

In [45]:
def h(w_j:np.array, x: np.array) -> np.array:
  print("x,w:", x.shape, w_j.shape)
  print("h:", (np.dot(x, w_j.reshape(1,))).shape)
  return sigmoid(np.dot(x, w_j.T))

# def logloss(w_j: np.array, x: np.array, b, y: np.array) -> np.float32:
#   # print(y * np.log(h(w_j, x)))
#   # print((1-y) * np.log(1 - h(w_j, x)))
#   # print(y * np.log(h(w_j, x)) + (1-y) * np.log(1 - h(w_j, x)))
#   # print(np.sum(y * np.log(h(w_j, x)) + (1-y) * np.log(1 - h(w_j, x))))
#   # print( - 1/n * np.sum(y * np.log(h(w_j, x)) + (1-y) * np.log(1 - h(w_j, x))))
#   return - 1/n * np.sum(y * np.log(g(w_j, x, b)) + (1-y) * np.log(1 - g(w_j, x,b)))

print(logloss(np.array([1,2, 2.9]), np.array([[1,3], [2,4], [2,3]], dtype=np.int32), np.array([1, 1.8, 3])))
# print(logloss(np.array([1.2,0,1.6,2,1.1,0,0.9]), np.array([-3, -2, -1, 0, 1,2,3]), np.array([-3, -3, -1, 0.1, 2,2,3])))
# print(logloss(np.array([1.2,0,1.6,2,1.1,0,0.9]), np.array([-3, -2, -1, 0, 1,2,3]), np.array([-4, -3, -1, 1, 20,2,3])))
# print(logloss(np.array([1.2,0,1.6,2,1.1,0,0.9]), np.array([-3, -2, -1, 0, 1,2,3]), np.array([-3, 18, -1.2, 100, 20,20,-30])))

x,w: (3, 2) (3,)


ValueError: shapes (3,2) and (3,) not aligned: 2 (dim 1) != 3 (dim 0)

Reminder:

* we are given a set of pairs $(x, y)$, where $x \in R^D$ is a vector of real numbers representing the features, and $y \in \{0,1\}$ is the target,
* for a given $x$ we model the probability of $y=1$ by $h(x):=g(w^Tx)$, where $g$ is the sigmoid function: $g(z) = \frac{1}{1+e^{-z}}$,
* to find the right $w$ we will optimize the so called logarithmic loss: $J(w) = -\frac{1}{n}\sum_{i=1}^n y_i \log{h(x_i)} + (1-y_i) \log{(1-h(x_i))}$,
* with the loss function in hand we can improve our guesses iteratively:
    * $w_j^{t+1} = w_j^{t} - \eta \cdot \frac{\partial J(w)}{\partial w_j}$

* we can end the process after some predefined number of epochs (or when the changes are no longer meaningful).

In [47]:
################################################################
# TODO: Implement logistic regression and compute its accuracy #
################################################################

w = np.random.random(size = (2))
b = 12
lr = 0.5 # step size

n_epochs = 40
losses = []

def ground_truth(points):
    return np.array([lin_rule(point) for point in points])

def predict(w, points, bias):
  return g(w, points, bias) > 0.5

def f(w, x, b):
  print("shapes in f:", w.shape, x.shape)
  print((np.dot(x, w.T) + b).shape)
  return np.dot(x, w.T) + b

def g(w, x, b):
  return sigmoid(f(w, x, b))

def logloss(w_j: np.array, x: np.array, b, y: np.array) -> np.float32:
  return - 1/n * np.sum(y * np.log(g(w_j, x, b)) + (1-y) * np.log(1 - g(w_j, x,b)))

def update(w, b, x, y):
  prediction = g(w,x,b)
  print(x.shape, prediction.shape, y.shape)
  w1 = w - (lr * 1/n * x * (prediction - y).reshape((n,1))).reshape((n,)) # comes from jacobian of logloss
  b1 = b - (lr * 1/n * 1 * (prediction - y).reshape((n,1))).reshape((n,))
  return w1, b1

for i in range(n_epochs):
    #############################
    # TODO: Fill in the details #
    #############################
    ### YOUR CODE BEGINS HERE ###
    w, b = update(w, b, X, y)
    loss = logloss(w, X, b, y)
    losses.append(loss)

    print(f'Iter: {i:>3} Loss: {loss:8.8f} w[:5]: {w[:5]}')
    print("Accuracy:", np.sum(predict(w, X, b) == y) / n)

shapes in f: (2,) (500, 2)
(500,)
(500, 2) (500,) (500,)


ValueError: cannot reshape array of size 1000 into shape (500,)

Let's visually asses our model. We can do this by using our estimates for $a,b,c$.

In [ ]:
#################################################################
# TODO: Pass your estimates for a,b,c to the get_y_fun function #
#################################################################
lin_fun2 = get_y_fun(...)

fig = px.scatter(x=X[:, 0], y=X[:, 1], color=list(map(str, y)))
x_range = [np.min(X[:, 0]), np.max(X[:, 1])]
fig.add_scatter(x=x_range, y=list(map(lin_fun, x_range)), name='ground truth border')
fig.add_scatter(x=x_range, y=list(map(lin_fun2, x_range)), name='estimated border')
fig.show()

Let's now complicate the things a little bit and make our next problem nonlinear.

In [ ]:
# Parameters of the ellipse
s1 = 1.
s2 = 2.
r = 0.75
m1 = 0.15
m2 = 0.125

# 0/1 mapping, checks whether we are inside the ellipse
def circle_rule(x, y, noise=0.):
    return 1 if s1 * (x - m1) ** 2 + s2 * (y - m2) ** 2 + noise < r ** 2 else 0

In [ ]:
# Training data

n = 500
range_points = 1

sigma = 0.1

X = range_points * 2 * (np.random.rand(n, 2) - 0.5)

y = [circle_rule(x, y, sigma * np.random.normal()) for x, y in X]

print(X[:10])
print(y[:10])

Let's plot the data.

In [ ]:
import plotly.graph_objects as go

fig = px.scatter(x=X[:, 0], y=X[:, 1], color=list(map(str, y)))

xgrid = np.arange(np.min(X[:, 0]), np.max(X[:, 0]), 0.003)
ygrid = np.arange(np.min(X[:, 1]), np.max(X[:, 1]), 0.003)
contour =  go.Contour(
        z=np.vectorize(circle_rule)(*np.meshgrid(xgrid, ygrid, indexing="ij")),
        x=xgrid,
        y=ygrid
    )
fig.add_trace(contour)
fig.show()

Now, let's train a logistic regression model to tackle this problem. Note that we now need a nonlinear decision boundary. You should obtain accuracy of at least 90%.

Hint:
<sub><sup><sub><sup><sub><sup>
Use feature engineering.
</sup></sub></sup></sub></sup></sub>

In [ ]:
################################################################
# TODO: Implement logistic regression and compute its accuracy #
################################################################

Let's visually asses our model.

Contrary to the previous scenario, converting our weights to parameters of the ground truth curve may not be straightforward. It's easier to just provide predictions for a set of points in $R^2$.

In [ ]:
h = .02

xgrid = np.arange(np.min(X[:, 0]), np.max(X[:, 0]), h)
ygrid = np.arange(np.min(X[:, 1]), np.max(X[:, 1]), h)

xx, yy = np.meshgrid(xgrid, ygrid, indexing="ij")
X_plot = np.c_[xx.ravel(), yy.ravel()]

print(X_plot.shape)

_X = np.concatenate([X_plot, X_plot**2], axis=1)

preds = logistic_regression(_w, _b, _X)
print(preds.shape)


In [ ]:
fig = px.scatter(x=X[:, 0], y=X[:, 1], color=list(map(str, y)))

xx, yy = np.meshgrid(xgrid, ygrid, indexing="ij")

contour = go.Contour(z=preds.reshape(len(xgrid), len(ygrid)), x=xgrid, y=ygrid)
fig.add_trace(contour)
fig.show()

<center><img src='https://drive.google.com/uc?id=1BXZ0u3562N_MqCLcekI-Ens77Kk4LpPm'></center>